<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Line_Disconnects_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
# Load the full dataset
fact_table = pd.read_csv('drive/My Drive/Churn/discos_truth_table.csv')

In [52]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,basic_CDMA,basic_non_CDMA,sp_android,sp_apple,sp_other
0,12/1/2016,2737,NaN,7809,8431,567.0
1,12/2/2016,2415,NaN,8392,9847,616.0
2,12/3/2016,2988,NaN,6277,7289,572.0
3,12/4/2016,3047,NaN,6300,7380,577.0
4,12/5/2016,119,NaN,517,586,24.0


In [0]:
# Split out indivudal datasets for forecasting
basic_CDMA = fact_table[['ds', 'basic_CDMA']].rename(columns = {'basic_CDMA': 'y'})
basic_non_CDMA = fact_table[['ds', 'basic_non_CDMA']].rename(columns = {'basic_non_CDMA': 'y'})
sp_android = fact_table[['ds', 'sp_android']].rename(columns = {'sp_android': 'y'})
sp_apple = fact_table[['ds', 'sp_apple']].rename(columns = {'sp_apple': 'y'})
sp_other = fact_table[['ds', 'sp_other']].rename(columns = {'sp_other': 'y'})

In [54]:
# Check the individual dataset format (testing only)
sp_apple.head()

,ds,y
0,12/1/2016,8431
1,12/2/2016,9847
2,12/3/2016,7289
3,12/4/2016,7380
4,12/5/2016,586


In [0]:
# Define the holidays
jan_bill_cycle = pd.DataFrame({
    'holiday': 'jan_bill_cycle',
    'ds': pd.to_datetime(['2018-01-05',
                          '2018-01-11',
                          '2018-01-14',
                          '2018-01-17',
                          '2018-01-24',
                          '2018-01-27',
                          '2019-01-29',
                          '2018-01-30',
                          '2018-01-31',
                          '2019-01-05',
                          '2019-01-11',
                          '2019-01-14',
                          '2019-01-17',
                          '2019-01-24',
                          '2019-01-27',
                          '2019-01-29',
                          '2019-01-30',
                          '2019-01-31',
                          '2020-01-05',
                          '2020-01-11',
                          '2020-01-14',
                          '2020-01-17',
                          '2020-01-24',
                          '2020-01-27',
                          '2020-01-29',
                          '2020-01-30',
                          '2020-01-31']),
    'lower_window': 0,
    'upper_window': 0,
})

feb_bill_cycle = pd.DataFrame({
    'holiday': 'feb_bill_cycle',
    'ds': pd.to_datetime(['2018-02-05',
                          '2018-02-11',
                          '2018-02-14',
                          '2018-02-17',
                          '2018-02-24',
                          '2018-02-27',
                          '2019-02-05',
                          '2019-02-11',
                          '2019-02-14',
                          '2019-02-17',
                          '2019-02-24',
                          '2019-02-27',
                          '2020-02-05',
                          '2020-02-11',
                          '2020-02-14',
                          '2020-02-17',
                          '2020-02-24',
                          '2020-02-27',
                          '2020-02-29']),
    'lower_window': 0,
    'upper_window': 0,
})

mar_bill_cycle = pd.DataFrame({
    'holiday': 'mar_bill_cycle',
    'ds': pd.to_datetime(['2018-03-05',
                          '2018-03-11',
                          '2018-03-14',
                          '2018-03-17',
                          '2018-03-24',
                          '2018-03-27',
                          '2019-03-29',
                          '2018-03-30',
                          '2018-03-31',
                          '2019-03-05',
                          '2019-03-11',
                          '2019-03-14',
                          '2019-03-17',
                          '2019-03-24',
                          '2019-03-27',
                          '2019-03-29',
                          '2019-03-30',
                          '2019-03-31',
                          '2020-03-05',
                          '2020-03-11',
                          '2020-03-14',
                          '2020-03-17',
                          '2020-03-24',
                          '2020-03-27',
                          '2020-03-29',
                          '2020-03-30',
                          '2020-03-31']),
    'lower_window': 0,
    'upper_window': 0,
})

apr_bill_cycle = pd.DataFrame({
    'holiday': 'apr_bill_cycle',
    'ds': pd.to_datetime(['2018-04-05',
                          '2018-04-11',
                          '2018-04-14',
                          '2018-04-17',
                          '2018-04-24',
                          '2018-04-27',
                          '2019-04-29',
                          '2018-04-30',
                          '2019-04-05',
                          '2019-04-11',
                          '2019-04-14',
                          '2019-04-17',
                          '2019-04-24',
                          '2019-04-27',
                          '2019-04-29',
                          '2019-04-30',
                          '2020-04-05',
                          '2020-04-11',
                          '2020-04-14',
                          '2020-04-17',
                          '2020-04-24',
                          '2020-04-27',
                          '2020-04-29',
                          '2020-04-30']),
    'lower_window': 0,
    'upper_window': 0,
})

may_bill_cycle = pd.DataFrame({
    'holiday': 'may_bill_cycle',
    'ds': pd.to_datetime(['2018-05-05',
                          '2018-05-11',
                          '2018-05-14',
                          '2018-05-17',
                          '2018-05-24',
                          '2018-05-27',
                          '2019-05-29',
                          '2018-05-30',
                          '2018-05-31',
                          '2019-05-05',
                          '2019-05-11',
                          '2019-05-14',
                          '2019-05-17',
                          '2019-05-24',
                          '2019-05-27',
                          '2019-05-29',
                          '2019-05-30',
                          '2019-05-31',
                          '2020-05-05',
                          '2020-05-11',
                          '2020-05-14',
                          '2020-05-17',
                          '2020-05-24',
                          '2020-05-27',
                          '2020-05-29',
                          '2020-05-30',
                          '2020-05-31']),
    'lower_window': 0,
    'upper_window': 0,
})

jun_bill_cycle = pd.DataFrame({
    'holiday': 'jun_bill_cycle',
    'ds': pd.to_datetime(['2018-06-05',
                          '2018-06-11',
                          '2018-06-14',
                          '2018-06-17',
                          '2018-06-24',
                          '2018-06-27',
                          '2019-06-29',
                          '2018-06-30',
                          '2019-06-05',
                          '2019-06-11',
                          '2019-06-14',
                          '2019-06-17',
                          '2019-06-24',
                          '2019-06-27',
                          '2019-06-29',
                          '2019-06-30',
                          '2020-06-05',
                          '2020-06-11',
                          '2020-06-14',
                          '2020-06-17',
                          '2020-06-24',
                          '2020-06-27',
                          '2020-06-29',
                          '2020-06-30']),
    'lower_window': 0,
    'upper_window': 0,
})

jul_bill_cycle = pd.DataFrame({
    'holiday': 'jul_bill_cycle',
    'ds': pd.to_datetime(['2018-07-05',
                          '2018-07-11',
                          '2018-07-14',
                          '2018-07-17',
                          '2018-07-24',
                          '2018-07-27',
                          '2019-07-29',
                          '2018-07-30',
                          '2018-07-31',
                          '2019-07-05',
                          '2019-07-11',
                          '2019-07-14',
                          '2019-07-17',
                          '2019-07-24',
                          '2019-07-27',
                          '2019-07-29',
                          '2019-07-30',
                          '2019-07-31',
                          '2020-07-05',
                          '2020-07-11',
                          '2020-07-14',
                          '2020-07-17',
                          '2020-07-24',
                          '2020-07-27',
                          '2020-07-29',
                          '2020-07-30',
                          '2020-07-31']),
    'lower_window': 0,
    'upper_window': 0,
})

aug_bill_cycle = pd.DataFrame({
    'holiday': 'aug_bill_cycle',
    'ds': pd.to_datetime(['2018-08-05',
                          '2018-08-11',
                          '2018-08-14',
                          '2018-08-17',
                          '2018-08-24',
                          '2018-08-27',
                          '2019-08-29',
                          '2018-08-30',
                          '2018-08-31',
                          '2019-08-05',
                          '2019-08-11',
                          '2019-08-14',
                          '2019-08-17',
                          '2019-08-24',
                          '2019-08-27',
                          '2019-08-29',
                          '2019-08-30',
                          '2019-08-31',
                          '2020-08-05',
                          '2020-08-11',
                          '2020-08-14',
                          '2020-08-17',
                          '2020-08-24',
                          '2020-08-27',
                          '2020-08-29',
                          '2020-08-30',
                          '2020-08-31']),
    'lower_window': 0,
    'upper_window': 0,
})

sep_bill_cycle = pd.DataFrame({
    'holiday': 'sep_bill_cycle',
    'ds': pd.to_datetime(['2018-09-05',
                          '2018-09-11',
                          '2018-09-14',
                          '2018-09-17',
                          '2018-09-24',
                          '2018-09-27',
                          '2019-09-29',
                          '2018-09-30',
                          '2019-09-05',
                          '2019-09-11',
                          '2019-09-14',
                          '2019-09-17',
                          '2019-09-24',
                          '2019-09-27',
                          '2019-09-29',
                          '2019-09-30',
                          '2020-09-05',
                          '2020-09-11',
                          '2020-09-14',
                          '2020-09-17',
                          '2020-09-24',
                          '2020-09-27',
                          '2020-09-29',
                          '2020-09-30']),
    'lower_window': 0,
    'upper_window': 0,
})

oct_bill_cycle = pd.DataFrame({
    'holiday': 'oct_bill_cycle',
    'ds': pd.to_datetime(['2018-10-05',
                          '2018-10-11',
                          '2018-10-14',
                          '2018-10-17',
                          '2018-10-24',
                          '2018-10-27',
                          '2019-10-29',
                          '2018-10-30',
                          '2018-10-31',
                          '2019-10-05',
                          '2019-10-11',
                          '2019-10-14',
                          '2019-10-17',
                          '2019-10-24',
                          '2019-10-27',
                          '2019-10-29',
                          '2019-10-30',
                          '2019-10-31',
                          '2020-10-05',
                          '2020-10-11',
                          '2020-10-14',
                          '2020-10-17',
                          '2020-10-24',
                          '2020-10-27',
                          '2020-10-29',
                          '2020-10-30',
                          '2020-10-31']),
    'lower_window': 0,
    'upper_window': 0,
})

nov_bill_cycle = pd.DataFrame({
    'holiday': 'nov_bill_cycle',
    'ds': pd.to_datetime(['2018-11-05',
                          '2018-11-11',
                          '2018-11-14',
                          '2018-11-17',
                          '2018-11-24',
                          '2018-11-27',
                          '2019-11-29',
                          '2018-11-30',
                          '2019-11-05',
                          '2019-11-11',
                          '2019-11-14',
                          '2019-11-17',
                          '2019-11-24',
                          '2019-11-27',
                          '2019-11-29',
                          '2019-11-30',
                          '2020-11-05',
                          '2020-11-11',
                          '2020-11-14',
                          '2020-11-17',
                          '2020-11-24',
                          '2020-11-27',
                          '2020-11-29',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

dec_bill_cycle = pd.DataFrame({
    'holiday': 'dec_bill_cycle',
    'ds': pd.to_datetime(['2018-12-05',
                          '2018-12-11',
                          '2018-12-14',
                          '2018-12-17',
                          '2018-12-24',
                          '2018-12-27',
                          '2019-12-29',
                          '2018-12-30',
                          '2018-12-31',
                          '2019-12-05',
                          '2019-12-11',
                          '2019-12-14',
                          '2019-12-17',
                          '2019-12-24',
                          '2019-12-27',
                          '2019-12-29',
                          '2019-12-30',
                          '2019-12-31',
                          '2020-12-05',
                          '2020-12-11',
                          '2020-12-14',
                          '2020-12-17',
                          '2020-12-24',
                          '2020-12-27',
                          '2020-12-29',
                          '2020-12-30',
                          '2020-12-31']),
    'lower_window': 0,
    'upper_window': 0,
})

In [0]:
holidays = pd.concat((jan_bill_cycle,
                      feb_bill_cycle,
                      mar_bill_cycle,
                      apr_bill_cycle,
                      may_bill_cycle,
                      jun_bill_cycle,
                      jul_bill_cycle,
                      aug_bill_cycle,
                      sep_bill_cycle,
                      oct_bill_cycle,
                      nov_bill_cycle,
                      dec_bill_cycle))

In [114]:
# Define and fit the models
basic_CDMA_model = Prophet(holidays = holidays).fit(basic_CDMA)
basic_non_CDMA_model = Prophet(holidays = holidays).fit(basic_non_CDMA)
sp_android_model = Prophet(holidays = holidays).fit(sp_android)
sp_apple_model = Prophet(holidays = holidays).fit(sp_apple)
sp_other_model = Prophet(holidays = holidays).fit(sp_other)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [0]:
# NUmber of periods into the future to forecast
n = 29

In [0]:
# Create dataframes to hold predictions
basic_CDMA_future = basic_CDMA_model.make_future_dataframe(periods = n)
basic_non_CDMA_future = basic_non_CDMA_model.make_future_dataframe(periods = n)
sp_android_future = sp_android_model.make_future_dataframe(periods = n)
sp_apple_future = sp_apple_model.make_future_dataframe(periods = n)
sp_other_future = sp_other_model.make_future_dataframe(periods = n)

In [0]:
# Make predictions
basic_CDMA_forecast = basic_CDMA_model.predict(basic_CDMA_future)
basic_non_CDMA_forecast = basic_non_CDMA_model.predict(basic_non_CDMA_future)
sp_android_forecast = sp_android_model.predict(sp_android_future)
sp_apple_forecast = sp_apple_model.predict(sp_apple_future)
sp_other_forecast = sp_other_model.predict(sp_other_future)

In [0]:
# Get the datestamp and prediction columns
basic_CDMA_forecast_subset = basic_CDMA_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Basic_CDMA'})
basic_non_CDMA_forecast_subset = basic_non_CDMA_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Basic_non_CDMA'})
sp_android_forecast_subset = sp_android_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'sp_Android'})
sp_apple_forecast_subset = sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'sp_Apple'})
sp_other_forecast_subset = sp_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'sp_Other'})

In [119]:
# Check the format of the subsetted forecasts (testing only)
basic_non_CDMA_forecast_subset.tail()

,ds,Basic_non_CDMA
1181,2020-02-25,140.679850
1182,2020-02-26,137.009215
1183,2020-02-27,100.473040
1184,2020-02-28,133.784151
1185,2020-02-29,103.030357


In [0]:
# Combine the forecasts into one table
discos_pred_table = basic_CDMA_forecast_subset
discos_pred_table.insert(2, 'Basic_non_CDMA', basic_non_CDMA_forecast_subset['Basic_non_CDMA'])
discos_pred_table.insert(3, 'sp_Android', sp_android_forecast_subset['sp_Android'])
discos_pred_table.insert(4, 'sp_Apple', sp_apple_forecast_subset['sp_Apple'])
discos_pred_table.insert(5, 'sp_Other', sp_other_forecast_subset['sp_Other'])

In [121]:
# Check the format of the final predictions table (testing only)
discos_pred_table.tail()

,ds,Basic_CDMA,Basic_non_CDMA,sp_Android,sp_Apple,sp_Other
1181,2020-02-25,1576.258860,140.679850,6096.530481,9122.969129,61.081733
1182,2020-02-26,1634.408540,137.009215,6208.023278,9291.947056,79.894926
1183,2020-02-27,-180.675165,100.473040,1606.290143,2939.255741,-66.041419
1184,2020-02-28,1549.518374,133.784151,5993.090039,9004.975373,71.676424
1185,2020-02-29,-248.614161,103.030357,1439.629784,2687.091134,-78.624524


In [0]:
# Save the predictions table to a csv file
discos_pred_table.to_csv('drive/My Drive/Churn/discos_pred_table.csv')